In [2]:
print("hello world!")

hello world!


In [3]:
!python --version

Python 3.10.14


In [5]:
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')


'520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu'

In [9]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [10]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

1.12.0-cpu: Pulling from sagemaker-tensorflow-serving
Digest: sha256:026c2a51a2e9e258481b17c110f9c9640dc010bf71463d4bd31e925ced7c6a49
Status: Image is up to date for 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu
